In [3]:
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, BatchNormalization, LSTM, Dense, concatenate, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50

import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cv2
import json

tf.__version__

'2.1.0'

# 1. Define loss & base network
- 두 가지 loss 함수
    - `triplet_loss`
        - 출처: https://thelonenutblog.wordpress.com/2017/12/18/what-siamese-dreams-are-made-of/
    - `lossless_triplet_loss`
        - 출처: https://towardsdatascience.com/lossless-triplet-loss-7e932f990b24

In [4]:
def lossless_triplet_loss(y_true, y_pred, N=128, beta=128, epsilon=1e-8):
    """
    Implementation of the triplet loss function
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    N  --  The number of dimension 
    beta -- The scaling factor, N is recommended
    epsilon -- The Epsilon value to prevent ln(0)
    
    
    Returns:
    loss -- real number, value of the loss
    """
    anchor = tf.convert_to_tensor(y_pred[:,0:N])
    positive = tf.convert_to_tensor(y_pred[:,N:N*2]) 
    negative = tf.convert_to_tensor(y_pred[:,N*2:N*3])
    
    # distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),1)
    # distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),1)
    
    #Non Linear Values  
    
    # -ln(-x/N+1)
    pos_dist = -tf.math.log(-tf.divide((pos_dist),beta)+1+epsilon)
    neg_dist = -tf.math.log(-tf.divide((N-neg_dist),beta)+1+epsilon)
    
    # compute loss
    loss = neg_dist + pos_dist
    
    
    return loss

In [5]:
def res_base_network(in_dims, out_dims, activation='sigmoid'):
    model = Sequential()
    
    model.add(ResNet50(
    include_top=False, weights='imagenet', input_shape=in_dims, pooling='avg', classes=1000))

    model.add(Dense(1024, activation = 'relu'))
    model.add(Dense(out_dims, activation = activation))

    return model

# 2. Define input generator

In [6]:
train_data_dir = '../train'
val_data_dir = '../val'

In [7]:
os.listdir(train_data_dir)

['(5598)Zippered_Faux_Leather_Joggers',
 '(267)Asymmetrical_Open-Front_Blazer',
 '(2166)Flat-Front_Skinny_Jeans',
 '(1871)Elephant_Paisley_Print_Dress',
 '(4096)Quilted_Dolphin_Shorts',
 '(1052)Clean_Wash_Slim-Leg_Jeans',
 '(1573)Denim_Daze_Ripped_Shorts',
 '(1095)Collarless_Moto_Jacket',
 '(32)Abstract_Chevron_Print_Shorts',
 '(4495)Side-Slit_Maxi_Dress',
 '(1587)Denim_Utility_Jumpsuit',
 '(4231)Ripped_Skinny_Jeans',
 '(4259)Rose_Print_Chiffon_Dress',
 '(4329)Rugby_Stripe_Sweater',
 '(409)Belted_Daisy_Print_Dress',
 '(1131)Colorblock_Yoga_Shorts',
 '(4178)Rhinestoned___Beaded_Blouse',
 '(5347)Tweed_Boyfriend_Blazer',
 '(781)Cascading_Layered_Shift_Dress',
 '(802)Charlie_Brown_Print_Tank',
 '(5307)Tropical_Print_Dress',
 '(2435)French_Terry_Tee',
 '(1428)Cropped_Trench_Coat',
 '(1194)Contrast_Trim_Henley',
 '(4944)Striped_Swim_Trunks',
 '(4887)Striped_Linen_Ringer_Tee',
 '(368)Beaded_Geo-Embroidered_Halter_Top',
 '(254)Art_Nouveau_Floral_Print_Top',
 '(4070)Polka_Dot_Woven_Shorts',
 '(

In [8]:
directory = os.listdir(train_data_dir)

In [9]:
len(directory)

5497

In [10]:
CLASS_DICT = {k:v for k, v in enumerate(directory)}
CLASS_DICT

{0: '(5598)Zippered_Faux_Leather_Joggers',
 1: '(267)Asymmetrical_Open-Front_Blazer',
 2: '(2166)Flat-Front_Skinny_Jeans',
 3: '(1871)Elephant_Paisley_Print_Dress',
 4: '(4096)Quilted_Dolphin_Shorts',
 5: '(1052)Clean_Wash_Slim-Leg_Jeans',
 6: '(1573)Denim_Daze_Ripped_Shorts',
 7: '(1095)Collarless_Moto_Jacket',
 8: '(32)Abstract_Chevron_Print_Shorts',
 9: '(4495)Side-Slit_Maxi_Dress',
 10: '(1587)Denim_Utility_Jumpsuit',
 11: '(4231)Ripped_Skinny_Jeans',
 12: '(4259)Rose_Print_Chiffon_Dress',
 13: '(4329)Rugby_Stripe_Sweater',
 14: '(409)Belted_Daisy_Print_Dress',
 15: '(1131)Colorblock_Yoga_Shorts',
 16: '(4178)Rhinestoned___Beaded_Blouse',
 17: '(5347)Tweed_Boyfriend_Blazer',
 18: '(781)Cascading_Layered_Shift_Dress',
 19: '(802)Charlie_Brown_Print_Tank',
 20: '(5307)Tropical_Print_Dress',
 21: '(2435)French_Terry_Tee',
 22: '(1428)Cropped_Trench_Coat',
 23: '(1194)Contrast_Trim_Henley',
 24: '(4944)Striped_Swim_Trunks',
 25: '(4887)Striped_Linen_Ringer_Tee',
 26: '(368)Beaded_Geo-E

In [11]:
gen = ImageDataGenerator(rescale=1./255)
num_classes = len(CLASS_DICT)
num_classes

5497

In [12]:
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [13]:
def input_triplet_gen(gen, data_dir, num_classes, batch_size):
    
    while True:
        anchor_label, neg_label = random.sample(range(0, num_classes), 2)
        
        POS_DIR = os.path.join(data_dir, CLASS_DICT[anchor_label])
        NEG_DIR = os.path.join(data_dir, CLASS_DICT[neg_label])
        
        POS_filecount = len(os.listdir(os.path.join(POS_DIR, POS_DIR.split(os.path.sep)[-1])))    #POS_DIR 파일 개수 (뎁스 2번 들어가야해서 os.path.join~ 해주기)
        NEG_filecount = len(os.listdir(os.path.join(NEG_DIR, NEG_DIR.split(os.path.sep)[-1])))   #NEG_DIR 파일 개수
        
        if  POS_filecount < batch_size or NEG_filecount < batch_size:    #파일 개수가 batch_size 미만이면 처음으로 돌아가서 클래스 다시 선택
            continue

        anchor_gen = gen.flow_from_directory(POS_DIR, 
                                             target_size=(224, 224),
                                             batch_size=batch_size,
                                             color_mode='rgb')

        pos_gen = gen.flow_from_directory(POS_DIR,
                                          target_size=(224, 224),
                                          batch_size=batch_size,
                                          color_mode='rgb')

        neg_gen = gen.flow_from_directory(NEG_DIR, 
                                          target_size=(224, 224),
                                          batch_size=batch_size,
                                          color_mode='rgb')
    
#         print("Anchor: {} , Neg: {}".format(anchor_label, neg_label))
        X1i = anchor_gen.next()
        X2i = pos_gen.next()
        X3i = neg_gen.next()
        
        yield [X1i[0], X2i[0], X3i[0]], X1i[1]

# 3. Create model

In [14]:
in_dims = (224, 224, 3)    # (28, 28, 1)
out_dims = 128

# Create the 3 inputs
anchor_in = Input(shape=in_dims, name='anchor')
pos_in = Input(shape=in_dims, name='positive')
neg_in = Input(shape=in_dims, name='negative')

# with tf.compat.v1.Session(config=config):
    # Share base network with the 3 inputs
base_network = res_base_network(in_dims, out_dims)
anchor_out = base_network(anchor_in)
pos_out = base_network(pos_in)
neg_out = base_network(neg_in)
    
merged_vector = concatenate([anchor_out, pos_out, neg_out], axis=-1)

# Define the trainable model
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)
model.compile(optimizer=Adam(learning_rate=0.0001, clipnorm=5.0),
                    loss=lossless_triplet_loss)

In [15]:
base_network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
Total params: 25,817,088
Trainable params: 25,763,968
Non-trainable params: 53,120
_________________________________________________________________


In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
positive (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
negative (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128)          25817088    anchor[0][0]                     
                                                                 positive[0][0]               

In [16]:
# plot_model(base_network, to_file='detectron_base.png', show_shapes=True, show_layer_names=True)

# 4. 이어서 학습하기

In [17]:
# #weights 로드
# model_weights_dir = './trained/resnet_detectron/000001-1.0883.h5'
# model.load_weights(model_weights_dir)

In [18]:
# model.summary()

# 4. Train

In [17]:
batch_size = 16

In [18]:
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='./trained/resnet_detectron_val/{epoch:06d}-{loss:.4f}.h5')]

In [ ]:
# Training the model
H = model.fit(input_triplet_gen(gen, train_data_dir, num_classes, batch_size),
                        steps_per_epoch = 1000, 
                        epochs=1000, 
                        validation_data = input_triplet_gen(gen, val_data_dir, num_classes, batch_size),
                        callbacks = my_callbacks) 

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']


In [ ]:
# 서환
# Training the model
test_set = ImageDataGenerator(rescale = 1./255).flow_from_directory('../val',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'binary')


H = model.fit_generator(input_triplet_gen(gen, train_data_dir, num_classes, batch_size),
                        steps_per_epoch = 30, 
                        epochs=1000, 
                        validation_data = test_set,
                        callbacks = my_callbacks) 

In [ ]:
#결과 확인
plt.plot(H.history['loss'])
# plt.savefig('./CNN/loss_graph.png')
# plt.legend(['loss'], loc = 'upper left')
plt.show()

# 5. Save model & weights

In [ ]:
model_dir = './trained/0527_10epoch_1000steps_model.json'
model_weights_dir = './trained/0527_10epoch_1000steps_model.h5'
base_dir = './trained/0527_10epoch_1000steps_base.json'
base_weights_dir = './trained/0527_10epoch_1000steps_base.h5'

In [ ]:
#Save model
model_json = model.to_json()
with open(model_dir, "w") as json_file : 
    json_file.write(model_json)

#Save weights
model.save_weights(model_weights_dir)
print("Saved model to disk")

In [ ]:
#Save base network
base_json = base_network.to_json()
with open(base_dir, "w") as json_file : 
    json_file.write(base_json)

#Save weights
base_network.save_weights(base_weights_dir)
print("Saved base network to disk")

In [ ]:
#Load base network & weights
from keras.models import model_from_json
json_file = open(base_dir, "r")
loaded_base_json = json_file.read()
json_file.close()
loaded_base_network = model_from_json(loaded_base_json)
loaded_base_network.load_weights(base_weights_dir)
print("Loaded model from disk")

In [ ]:
model_dir = './trained/0527_10epoch_1000steps_model.json'
model_weights_dir = './trained/0527_10epoch_1000steps_model.h5'
model_weights_dir = './trained/0527_10epoch_1000steps_model.h5'

In [ ]:
#Load base network & weights
from keras.models import model_from_json
json_file = open(model_dir, "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(model_weights_dir)
print("Loaded model from disk")

# 6. Application

In [ ]:
def item_to_encoding(item, model):
    return model.predict(np.array([item]))

## 6.1 Load test data

In [ ]:
test_dir = '../img/img_moved2'    #클래스 5개만 있는 테스트 디렉토리

In [ ]:
test_batch_size = 2000

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
                        data_dir,  # this is the target directory
                        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
                        batch_size=test_batch_size)

batch = test_generator.next()
images = batch[0]
labels = batch[1]

# images = images.reshape(-1, 150, 150)
labels = np.array([np.argmax(x) for x in labels])

In [ ]:
len(set(labels))

In [ ]:
    while True:
        anchor_label, neg_label = random.sample(range(0, num_classes), 2)
        
        POS_DIR = os.path.join(data_dir, CLASS_DICT[anchor_label])
        NEG_DIR = os.path.join(data_dir, CLASS_DICT[neg_label])
        
        POS_filecount = len(os.listdir(os.path.join(POS_DIR, POS_DIR.split(os.path.sep)[-1])))    #POS_DIR 파일 개수 (뎁스 2번 들어가야해서 os.path.join~ 해주기)
        NEG_filecount = len(os.listdir(os.path.join(NEG_DIR, NEG_DIR.split(os.path.sep)[-1])))   #NEG_DIR 파일 개수
        
        if  POS_filecount < batch_size or NEG_filecount < batch_size:    #파일 개수가 batch_size 미만이면 처음으로 돌아가서 클래스 다시 선택
            continue

        anchor_gen = gen.flow_from_directory(POS_DIR, 
                                                      target_size=(150, 150),
                                                     batch_size=batch_size)

        pos_gen = gen.flow_from_directory(POS_DIR, 
                                                      target_size=(150, 150),
                                                     batch_size=batch_size)

        neg_gen = gen.flow_from_directory(NEG_DIR, 
                                                      target_size=(150, 150),
                                                     batch_size=batch_size)
    
#         print("Anchor: {} , Neg: {}".format(anchor_label, neg_label))
        X1i = anchor_gen.next()
        X2i = pos_gen.next()
        X3i = neg_gen.next()
        
        yield [X1i[0], X2i[0], X3i[0]], X1i[1]

In [ ]:
def get_triplets(x, y, sample_size):
    a = []
    p = []
    n = []
    
    num_classes = len(set(y))
    for _ in range(sample_size):
        num1, num2 = random.sample(set(y), 2)
        positive_pool = x[np.where(y == num1)[0]]
        negative_pool = x[np.where(y == num2)[0]]
        
        anchor, positive = random.sample(positive_pool, 2)
        negative = negative_pool[random.randint(0, len(negative_pool)-1)]
        
        a.append(anchor)
        p.append(positive)
        n.append(negative)
        
    return np.asarray(a), np.asarray(p), np.asarray(n)

In [ ]:
sample_size = 100
t_a, t_p, t_n = get_triplets(images, labels, sample_size)

## 6.2 triplet test

In [ ]:
count_true = 0
for k in range(sample_size):
    anchor_encoding = item_to_encoding(t_a[k], base_network)
    pos_encoding = item_to_encoding(t_p[k], base_network)
    neg_encoding = item_to_encoding(t_n[k], base_network)

#     anchor_pos_dist = np.linalg.norm(anchor_encoding-pos_encoding)
#     anchor_neg_dist = np.linalg.norm(anchor_encoding-neg_encoding)

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor_encoding,pos_encoding)),1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor_encoding,neg_encoding)),1)
    
    # -ln(-x/N+1)
    anchor_pos_dist = -tf.math.log(-tf.divide((pos_dist),128)+1+1e-8)
    anchor_neg_dist = -tf.math.log(-tf.divide((128-neg_dist),128)+1+1e-8)

    cond = anchor_pos_dist < anchor_neg_dist
    
    if cond: count_true += 1
        
    else: 
        print('---------- wrong embedding : index %d -----------'%k)
        plt.imshow(np.hstack([t_a[k, :, :], t_p[k, :, :], t_n[k, :, :]]))
        plt.show()
        
print("true: {} false: {}".format(count_true, sample_size-count_true))
print("Accuracy: {}".format(count_true / sample_size))

## 6.3 Anchor test

In [ ]:
#일부만 테스트 + 이미지 출력
thresh = 0.1

test_size = 100

anchor = images[100]
anchor_encoding = item_to_encoding(anchor, base_network)

plt.figure(figsize=(300, 300))

cc = 3
rr = int(len(images[:test_size]) / cc)+1

plt.subplot( rr ,cc, 1)
plt.imshow(anchor, cmap = 'gray')
plt.xlabel("Anchor({})".format(labels[0]), fontsize=80)

correct_count = 0

for k in range(len(images[:test_size])):        
    i = images[k]          
    plt.subplot( rr ,cc, k+2)   # 몇 칼럼 몇 로우로 그래프 그릴지
    plt.imshow(i, cmap = 'gray')
    item_encoding = item_to_encoding(i, base_network)
    
    dist = tf.reduce_sum(tf.square(tf.subtract(anchor_encoding, item_encoding)),1)
    dist = -tf.math.log(-tf.divide((dist),128)+1+1e-8)
    dist = dist.numpy()[0]
    
    similar = True if dist < thresh else False
    SIM = "SIMILAR" if similar else "NOT SIMILAR"
    cond = labels[0] == labels[k]
    
    if (similar and cond) or (not similar and not cond ):
        c = "blue"
        correct_count += 1
    else: c = "red"
    
    plt.xlabel("{} true label: {} distance from anchor: {:.2f}".format(SIM, labels[k], dist),  color = c, fontsize=80)
    
print("true: {}, false: {}".format(correct_count, test_size-correct_count))
print("Accuracy: {}".format(correct_count / test_size))

In [ ]:
#전체 테스트
#이미지 많이 출력하면 커널 죽어서 이미지 출력은 주석처리
thresh = 0.1

anchor = images[0]
anchor_encoding = item_to_encoding(anchor, base_network)

# plt.figure(figsize=(150, 150))

# cc = 3
# rr = int(len(images[:50]) / cc)+1

# plt.subplot( rr ,cc, 1)
# plt.imshow(anchor, cmap = 'gray')
# plt.xlabel("Anchor({})".format(labels[0]), fontsize=80)

correct_count = 0

for k in range(len(images[:200])):        
    i = images[k]          
#     plt.subplot( rr ,cc, k+2)   # 몇 칼럼 몇 로우로 그래프 그릴지
#     plt.imshow(i, cmap = 'gray')
    item_encoding = item_to_encoding(i, base_network)
    
    dist = tf.reduce_sum(tf.square(tf.subtract(anchor_encoding, item_encoding)),1)
    dist = -tf.math.log(-tf.divide((dist),128)+1+1e-8)
    dist = dist.numpy()[0]
    
    similar = True if dist < thresh else False
    SIM = "SIMILAR" if similar else "NOT SIMILAR"
    cond = labels[0] == labels[k]
    
    if (similar and cond) or (not similar and not cond ):
        c = "blue"
        correct_count += 1
    else: c = "red"
    
#     plt.xlabel("{} true label: {} distance from anchor: {:.2f}".format(SIM, labels[k], dist),  color = c, fontsize=80)
    
print("true: {}, false: {}".format(correct_count, 200-correct_count))
print("Accuracy: {}".format(correct_count / 200))

## 6.3 Anchor-all test

In [ ]:
thresh = 0.1

anchor = images[0]
anchor_encoding = item_to_encoding(anchor, loaded_base_network)

# plt.figure(figsize=(150, 150))

# cc = 3
# rr = int(len(images[:50]) / cc)+1

# plt.subplot( rr ,cc, 1)
# plt.imshow(anchor, cmap = 'gray')
# plt.xlabel("Anchor({})".format(labels[0]), fontsize=80)

correct_count = 0

for k in range(len(images[:200])):        
    i = images[k]          
#     plt.subplot( rr ,cc, k+2)   # 몇 칼럼 몇 로우로 그래프 그릴지
#     plt.imshow(i, cmap = 'gray')
    item_encoding = item_to_encoding(i, loaded_base_network)
    
    dist = tf.reduce_sum(tf.square(tf.subtract(anchor_encoding, item_encoding)),1)
    dist = -tf.math.log(-tf.divide((dist),128)+1+1e-8)
    dist = dist.numpy()[0]
    
    similar = True if dist < thresh else False
    SIM = "SIMILAR" if similar else "NOT SIMILAR"
    cond = labels[0] == labels[k]
    
    if (similar and cond) or (not similar and not cond ):
        c = "blue"
        correct_count += 1
    else: c = "red"
    
#     plt.xlabel("{} true label: {} distance from anchor: {:.2f}".format(SIM, labels[k], dist),  color = c, fontsize=80)
    
print("true: {}, false: {}".format(correct_count, 200-correct_count))
print("Accuracy: {}".format(correct_count / 200))

## 6.4 Anchor-Positive test

In [ ]:
thresh = 0.1

anchor_index = 0     #테스트할 anchor의 인덱스

anchor = images[anchor_index]
anchor_label = labels[anchor_index]
anchor_encoding = item_to_encoding(anchor, base_network)

positive_images = images[np.where(labels == anchor_label)]     #anchor와 라벨이 같은 positive 이미지들
num_positive = len(positive_images)     #positive 이미지 개수

plt.figure(figsize=(150, 150))

cc = 3
rr = int(num_positive / cc)+1

plt.subplot( rr ,cc, 1)
plt.imshow(anchor, cmap = 'gray')
plt.xlabel("Anchor", fontsize=80)

correct_count = 0

for k in range(num_positive):        
    i = positive_images[k]          
    plt.subplot( rr ,cc, k+2)   # 몇 칼럼 몇 로우에 이미지 표시할지
    plt.imshow(i)
    item_encoding = item_to_encoding(i, base_network)
    
    dist = tf.reduce_sum(tf.square(tf.subtract(anchor_encoding, item_encoding)),1)
    dist = -tf.math.log(-tf.divide((dist),128)+1+1e-8)
    dist = dist.numpy()[0]
    
    similar = True if dist < thresh else False
    SIM = "SIMILAR" if similar else "NOT SIMILAR"
    
    if similar:
        c = "blue"
        correct_count += 1
    else: c = "red"
    
#     plt.xlabel("{} true label: {} distance from anchor: {:.2f}".format(SIM, labels[k], dist),  color = c, fontsize=80)
    
print("true: {}, false: {}".format(correct_count, num_positive-correct_count))
print("Accuracy: {}".format(correct_count / num_positive))

In [ ]:
def sub_dir(o_dir, n_dir):
    """
    Arguments:
    - o_dir: old directory. 뎁스 바꿔주고 싶은 이미지 폴더(루트)
    e.g. '../../img/mmfashion'
    
    -n_dir: new directory. 새로운 폴더명. 아무거나 상관없음
    e.g. '../../img/mmfashion_flow'
    
    Return:
    없음
    
    """
    dirlist=os.listdir(o_dir)
    for i in dirlist:
        if not os.path.exists(n_dir):
            os.mkdir(n_dir)
        if not os.path.exists(n_dir+'/'+i):
            os.mkdir(n_dir+'/'+i)
        shutil.move(o_dir+"/"+i,n_dir+'/'+i)
        
sub_dir( '../masked_moved',  '../masked_moved2')   